In [82]:
import pandas as pd
import yfinance as yf
from datetime import datetime

In [85]:
tickers = ['NVDA', 'SPY'] # As specify, Nnvdia stock and SPY ETF
num_expiries = 4 # Given next market closing day (Next Friday)
save_path = '../data/'

In [79]:
temp = 'NVDA'
yf_ticker = yf.Ticker(temp)
spot_price = yf_ticker.history(period="1d")['Close'].iloc[-1]
dividend_yield = yf_ticker.info.get('dividendYield', 0.0)

print(f"\n{temp} - Spot: {spot_price:.2f}, Dividend Yield: {dividend_yield}")

all_data = []

expiries = yf_ticker.options[:num_expiries]

for expiry in expiries:
    try:
        option_chain = yf_ticker.option_chain(expiry)
    except:
        print(f"Failed to fetch options for {temp} on {expiry}")
        continue

    for opt_type, df in [('call', option_chain.calls), ('put', option_chain.puts)]:
        df = df.copy()
        df['type'] = opt_type
        df['expiry'] = expiry
        df['spot'] = spot_price
        df['dividendYield'] = dividend_yield
        all_data.append(df)


NVDA - Spot: 109.67, Dividend Yield: 0.04


In [70]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # auto-expand width
pd.set_option('display.max_colwidth', None) # don't truncate long text

In [71]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [86]:
# Inspect Data added (sand box)
print(all_data[0].columns)
print(all_data[3][["contractSymbol","strike","expiry","type", "dividendYield"]])

Index(['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask',
       'change', 'percentChange', 'volume', 'openInterest',
       'impliedVolatility', 'inTheMoney', 'contractSize', 'currency', 'type',
       'expiry', 'spot', 'dividendYield'],
      dtype='object')
         contractSymbol  strike      expiry type  dividendYield
0    SPY250417P00260000   260.0  2025-04-17  put           1.28
1    SPY250417P00265000   265.0  2025-04-17  put           1.28
2    SPY250417P00270000   270.0  2025-04-17  put           1.28
3    SPY250417P00275000   275.0  2025-04-17  put           1.28
4    SPY250417P00280000   280.0  2025-04-17  put           1.28
..                  ...     ...         ...  ...            ...
190  SPY250417P00720000   720.0  2025-04-17  put           1.28
191  SPY250417P00750000   750.0  2025-04-17  put           1.28
192  SPY250417P00765000   765.0  2025-04-17  put           1.28
193  SPY250417P00810000   810.0  2025-04-17  put           1.28
194  SPY250417

In [87]:
# Inspect full Calls and Puts df
sliced_data = [df.iloc[:, 2:] for df in all_data]
for i, df in enumerate(sliced_data):
    print(f"\n📄 Sliced Option Chain {i+1}")
    print(df)


📄 Sliced Option Chain 1
     strike  lastPrice     bid     ask     change  percentChange  volume  \
0     375.0     138.99  130.40  132.05 -23.649994     -14.541314    70.0   
1     380.0     129.11  125.38  127.83 -33.069992     -20.390920     3.0   
2     395.0     125.87  109.65  112.97 -35.790000     -22.139057     4.0   
3     400.0     107.39  104.75  107.78 -68.490005     -38.941326    21.0   
4     410.0     142.58   95.01   98.32   0.000000       0.000000     NaN   
..      ...        ...     ...     ...        ...            ...     ...   
141   675.0       0.01    0.00    0.03   0.000000       0.000000     2.0   
142   680.0       0.02    0.00    0.03   0.000000       0.000000     1.0   
143   685.0       0.01    0.00    0.03   0.000000       0.000000   249.0   
144   690.0       0.01    0.00    0.03   0.000000       0.000000     NaN   
145   700.0       0.01    0.00    0.02   0.000000       0.000000     4.0   

     openInterest  impliedVolatility  inTheMoney contractSize 

In [88]:
for ticker in tickers:
    yf_ticker = yf.Ticker(ticker)
    spot_price = yf_ticker.history(period="1d")['Close'].iloc[-1]
    dividend_yield = yf_ticker.info.get('dividendYield', 0.0)

    print(f"\n{ticker} - Spot: {spot_price:.2f}, Dividend Yield: {dividend_yield}")

    all_data = []

    if ticker.upper() == 'SPY':
        expiries = ['2025-04-11', '2025-04-17', '2025-04-25', '2025-05-02']
    else:
        expiries = yf_ticker.options[:num_expiries]

    for expiry in expiries:
        try:
            option_chain = yf_ticker.option_chain(expiry)
        except:
            print(f"Failed to fetch options for {ticker} on {expiry}")
            continue

        for opt_type, df in [('call', option_chain.calls), ('put', option_chain.puts)]:
            df = df.copy() # ensures not editing the original DataFrame
            # Add sufficient Columns
            df['type'] = opt_type
            df['expiry'] = expiry # ensures if need to scale the data for increase reachness
            df['spot'] = spot_price # model Input as (S)
            df['dividendYield'] = dividend_yield # yield exist only for SPY, NVDA yield = 0.04
            all_data.append(df)
            
    options_df = pd.concat(all_data).reset_index(drop=True)
    filename = f"{save_path}options_{ticker.lower()}.csv"
    options_df.to_csv(filename, index=False)
    print(f"Saved: {filename}")


NVDA - Spot: 94.31, Dividend Yield: 0.04
Saved: ../data/options_nvda.csv

SPY - Spot: 505.28, Dividend Yield: 1.28
Saved: ../data/options_spy.csv


In [89]:
print("# of rows =" ,sum(len(df) for df in all_data))

# of rows = 1116
